# Imports

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.models import TextClassifier

from flair.data import TaggedCorpus
from flair.data_fetcher import  NLPTaskDataFetcher, NLPTask

import torch

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# Initialize Mongo database

In [4]:
from pymongo import MongoClient

client = MongoClient()

db = client['glvis_db']

# Extract hidden representations from flair's pretrained POS model

In [5]:
pos_tagger = SequenceTagger.load('pos')

2019-03-23 17:01:29,537 loading file /home/snie/.flair/models/en-pos-ontonotes-v0.2.pt


In [6]:
base_path='/home/snie/Workspace/glvis/models/data/conll/OntoNotes-5.0-NER-BIO/'

In [7]:
corpus: TaggedCorpus = NLPTaskDataFetcher.load_corpus(NLPTask.ONTONER, base_path=base_path)

2019-03-23 17:01:32,614 Reading data from /home/snie/Workspace/glvis/models/data/conll/OntoNotes-5.0-NER-BIO/ontoner
2019-03-23 17:01:32,614 Train: /home/snie/Workspace/glvis/models/data/conll/OntoNotes-5.0-NER-BIO/ontoner/onto.train.ner
2019-03-23 17:01:32,615 Dev: /home/snie/Workspace/glvis/models/data/conll/OntoNotes-5.0-NER-BIO/ontoner/onto.development.ner
2019-03-23 17:01:32,615 Test: None


In [8]:
pos_tagger

SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharLMEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_1): CharLMEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=53, bias=True)
)

In [9]:
tag_dictionary = corpus.make_tag_dictionary(tag_type='pos')

In [11]:
len(corpus.get_all_sentences())

84790

## First linear layer

In [10]:
db_col = db['flair_pos_embedding2nn']

In [16]:
db_col.drop()

In [15]:
for i, sentence in enumerate(corpus.get_all_sentences()):
    print(f'Start sentence {i}')
    
    # Define hook to get intermediate values
    hidden_states = torch.zeros(len(sentence), 1, 4096)
    def hook(m, i):
        hidden_states.copy_(i[0].data)
                                
    h = pos_tagger.embedding2nn.register_forward_pre_hook(hook)
                                
    pos_tagger.predict(sentence)
                                
    spans = sentence.get_spans('pos')
                                
    # Informaction to store: the named entities, their predicted labels, probabilities and hidden states
    # If there are multiple words for one entity, take the average value of hidden states
    # and record the number of words in the entity
    
    for span in spans:
        entry = {}
        entry['text'] = span.text
        entry['tag'] = span.tag
        entry['score'] = span.score
        entry['token_num'] = len(span.tokens)
        
        idx = span.tokens[0].idx - 1
        entry['embedding2nn'] = hidden_states[idx, :, :].squeeze().tolist()
        
        db_col.insert_one(entry)
    
    h.remove()
    
    print(f'Finish sentence {i}')

Start sentence 0
Finish sentence 0
Start sentence 1
Finish sentence 1
Start sentence 2
Finish sentence 2
Start sentence 3
Finish sentence 3
Start sentence 4
Finish sentence 4
Start sentence 5
Finish sentence 5
Start sentence 6
Finish sentence 6
Start sentence 7
Finish sentence 7
Start sentence 8
Finish sentence 8
Start sentence 9
Finish sentence 9
Start sentence 10
Finish sentence 10
Start sentence 11
Finish sentence 11
Start sentence 12
Finish sentence 12
Start sentence 13
Finish sentence 13
Start sentence 14
Finish sentence 14
Start sentence 15
Finish sentence 15
Start sentence 16
Finish sentence 16
Start sentence 17
Finish sentence 17
Start sentence 18
Finish sentence 18
Start sentence 19
Finish sentence 19
Start sentence 20
Finish sentence 20
Start sentence 21
Finish sentence 21
Start sentence 22
Finish sentence 22
Start sentence 23
Finish sentence 23
Start sentence 24
Finish sentence 24
Start sentence 25
Finish sentence 25
Start sentence 26
Finish sentence 26
Start sentence 27
Fin

Finish sentence 216
Start sentence 217
Finish sentence 217
Start sentence 218
Finish sentence 218
Start sentence 219
Finish sentence 219
Start sentence 220
Finish sentence 220
Start sentence 221
Finish sentence 221
Start sentence 222
Finish sentence 222
Start sentence 223
Finish sentence 223
Start sentence 224
Finish sentence 224
Start sentence 225
Finish sentence 225
Start sentence 226
Finish sentence 226
Start sentence 227
Finish sentence 227
Start sentence 228
Finish sentence 228
Start sentence 229
Finish sentence 229
Start sentence 230
Finish sentence 230
Start sentence 231
Finish sentence 231
Start sentence 232
Finish sentence 232
Start sentence 233
Finish sentence 233
Start sentence 234
Finish sentence 234
Start sentence 235
Finish sentence 235
Start sentence 236
Finish sentence 236
Start sentence 237
Finish sentence 237
Start sentence 238
Finish sentence 238
Start sentence 239
Finish sentence 239
Start sentence 240
Finish sentence 240
Start sentence 241
Finish sentence 241
Start

Finish sentence 428
Start sentence 429
Finish sentence 429
Start sentence 430
Finish sentence 430
Start sentence 431
Finish sentence 431
Start sentence 432
Finish sentence 432
Start sentence 433
Finish sentence 433
Start sentence 434
Finish sentence 434
Start sentence 435
Finish sentence 435
Start sentence 436
Finish sentence 436
Start sentence 437
Finish sentence 437
Start sentence 438
Finish sentence 438
Start sentence 439
Finish sentence 439
Start sentence 440
Finish sentence 440
Start sentence 441
Finish sentence 441
Start sentence 442
Finish sentence 442
Start sentence 443
Finish sentence 443
Start sentence 444
Finish sentence 444
Start sentence 445
Finish sentence 445
Start sentence 446
Finish sentence 446
Start sentence 447
Finish sentence 447
Start sentence 448
Finish sentence 448
Start sentence 449
Finish sentence 449
Start sentence 450
Finish sentence 450
Start sentence 451
Finish sentence 451
Start sentence 452
Finish sentence 452
Start sentence 453
Finish sentence 453
Start

Finish sentence 639
Start sentence 640
Finish sentence 640
Start sentence 641
Finish sentence 641
Start sentence 642
Finish sentence 642
Start sentence 643
Finish sentence 643
Start sentence 644
Finish sentence 644
Start sentence 645
Finish sentence 645
Start sentence 646
Finish sentence 646
Start sentence 647
Finish sentence 647
Start sentence 648
Finish sentence 648
Start sentence 649
Finish sentence 649
Start sentence 650
Finish sentence 650
Start sentence 651
Finish sentence 651
Start sentence 652
Finish sentence 652
Start sentence 653
Finish sentence 653
Start sentence 654
Finish sentence 654
Start sentence 655
Finish sentence 655
Start sentence 656
Finish sentence 656
Start sentence 657
Finish sentence 657
Start sentence 658
Finish sentence 658
Start sentence 659
Finish sentence 659
Start sentence 660
Finish sentence 660
Start sentence 661
Finish sentence 661
Start sentence 662
Finish sentence 662
Start sentence 663
Finish sentence 663
Start sentence 664
Finish sentence 664
Start

Finish sentence 852
Start sentence 853
Finish sentence 853
Start sentence 854
Finish sentence 854
Start sentence 855
Finish sentence 855
Start sentence 856
Finish sentence 856
Start sentence 857
Finish sentence 857
Start sentence 858
Finish sentence 858
Start sentence 859
Finish sentence 859
Start sentence 860
Finish sentence 860
Start sentence 861
Finish sentence 861
Start sentence 862
Finish sentence 862
Start sentence 863
Finish sentence 863
Start sentence 864
Finish sentence 864
Start sentence 865
Finish sentence 865
Start sentence 866
Finish sentence 866
Start sentence 867
Finish sentence 867
Start sentence 868
Finish sentence 868
Start sentence 869
Finish sentence 869
Start sentence 870
Finish sentence 870
Start sentence 871
Finish sentence 871
Start sentence 872
Finish sentence 872
Start sentence 873
Finish sentence 873
Start sentence 874
Finish sentence 874
Start sentence 875
Finish sentence 875
Start sentence 876
Finish sentence 876
Start sentence 877
Finish sentence 877
Start

...

Finish sentence 15880
Start sentence 15881
Finish sentence 15881
Start sentence 15882
Finish sentence 15882
Start sentence 15883
Finish sentence 15883
Start sentence 15884
Finish sentence 15884
Start sentence 15885
Finish sentence 15885
Start sentence 15886
Finish sentence 15886
Start sentence 15887
Finish sentence 15887
Start sentence 15888
Finish sentence 15888
Start sentence 15889
Finish sentence 15889
Start sentence 15890
Finish sentence 15890
Start sentence 15891
Finish sentence 15891
Start sentence 15892
Finish sentence 15892
Start sentence 15893
Finish sentence 15893
Start sentence 15894
Finish sentence 15894
Start sentence 15895
Finish sentence 15895
Start sentence 15896
Finish sentence 15896
Start sentence 15897
Finish sentence 15897
Start sentence 15898
Finish sentence 15898
Start sentence 15899
Finish sentence 15899
Start sentence 15900
Finish sentence 15900
Start sentence 15901
Finish sentence 15901
Start sentence 15902
Finish sentence 15902
Start sentence 15903
Finish sent

Finish sentence 16071
Start sentence 16072
Finish sentence 16072
Start sentence 16073
Finish sentence 16073
Start sentence 16074
Finish sentence 16074
Start sentence 16075
Finish sentence 16075
Start sentence 16076
Finish sentence 16076
Start sentence 16077
Finish sentence 16077
Start sentence 16078
Finish sentence 16078
Start sentence 16079
Finish sentence 16079
Start sentence 16080
Finish sentence 16080
Start sentence 16081
Finish sentence 16081
Start sentence 16082
Finish sentence 16082
Start sentence 16083
Finish sentence 16083
Start sentence 16084
Finish sentence 16084
Start sentence 16085
Finish sentence 16085
Start sentence 16086
Finish sentence 16086
Start sentence 16087
Finish sentence 16087
Start sentence 16088
Finish sentence 16088
Start sentence 16089
Finish sentence 16089
Start sentence 16090
Finish sentence 16090
Start sentence 16091
Finish sentence 16091
Start sentence 16092
Finish sentence 16092
Start sentence 16093
Finish sentence 16093
Start sentence 16094
Finish sent

Finish sentence 16262
Start sentence 16263
Finish sentence 16263
Start sentence 16264
Finish sentence 16264
Start sentence 16265
Finish sentence 16265
Start sentence 16266
Finish sentence 16266
Start sentence 16267
Finish sentence 16267
Start sentence 16268
Finish sentence 16268
Start sentence 16269
Finish sentence 16269
Start sentence 16270
Finish sentence 16270
Start sentence 16271
Finish sentence 16271
Start sentence 16272
Finish sentence 16272
Start sentence 16273
Finish sentence 16273
Start sentence 16274
Finish sentence 16274
Start sentence 16275
Finish sentence 16275
Start sentence 16276
Finish sentence 16276
Start sentence 16277
Finish sentence 16277
Start sentence 16278
Finish sentence 16278
Start sentence 16279
Finish sentence 16279
Start sentence 16280
Finish sentence 16280
Start sentence 16281
Finish sentence 16281
Start sentence 16282
Finish sentence 16282
Start sentence 16283
Finish sentence 16283
Start sentence 16284
Finish sentence 16284
Start sentence 16285
Finish sent

Finish sentence 16453
Start sentence 16454
Finish sentence 16454
Start sentence 16455
Finish sentence 16455
Start sentence 16456
Finish sentence 16456
Start sentence 16457
Finish sentence 16457
Start sentence 16458
Finish sentence 16458
Start sentence 16459
Finish sentence 16459
Start sentence 16460
Finish sentence 16460
Start sentence 16461
Finish sentence 16461
Start sentence 16462
Finish sentence 16462
Start sentence 16463
Finish sentence 16463
Start sentence 16464
Finish sentence 16464
Start sentence 16465
Finish sentence 16465
Start sentence 16466
Finish sentence 16466
Start sentence 16467
Finish sentence 16467
Start sentence 16468
Finish sentence 16468
Start sentence 16469
Finish sentence 16469
Start sentence 16470
Finish sentence 16470
Start sentence 16471
Finish sentence 16471
Start sentence 16472
Finish sentence 16472
Start sentence 16473
Finish sentence 16473
Start sentence 16474
Finish sentence 16474
Start sentence 16475
Finish sentence 16475
Start sentence 16476
Finish sent

KeyboardInterrupt: 

## Last linear layer

In [5]:
db_col = db['flair_pos_linear']

In [ ]:
for i, sentence in enumerate(corpus.get_all_sentences()):
    print(f'Start sentence {i}')
    
    # Define hook to get intermediate values
    hidden_states = torch.zeros(len(sentence), 1, 512)
    def hook(m, i):
        hidden_states.copy_(i[0].data)
                                
    h = pos_tagger.linear.register_forward_pre_hook(hook)
                                
    pos_tagger.predict(sentence)
                                
    spans = sentence.get_spans('pos')
                                
    # Informaction to store: the named entities, their predicted labels, probabilities and hidden states
    # If there are multiple words for one entity, take the average value of hidden states
    # and record the number of words in the entity
    
    for span in spans:
        entry = {}
        entry['text'] = span.text
        entry['tag'] = span.tag
        entry['score'] = span.score
        entry['token_num'] = len(span.tokens)
        
        idx = span.tokens[0].idx - 1
        entry['linear_layer_state'] = hidden_states[idx, :, :].squeeze().tolist()
        
        db_col.insert_one(entry)
    
    h.remove()
    
    print(f'Finish sentence {i}')